In [7]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('databases/merged_gtfs.db')
gtfs_names = ['routes', 'stop_times', 'stops', 'transfers', 'trips']

for name in gtfs_names:
    df = pd.read_csv(f"gtfs/{name}.txt")
    df.to_sql(name, conn, if_exists='replace', index=False)

# Close the connection
conn.close()

/tmp/ipykernel_37501/2957384310.py:9: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"gtfs/{name}.txt")


In [9]:
import sqlite3

def test_query(db_path: str, query: str):
    """
    Test function to run a SQL query on the specified SQLite database.

    :param db_path: Path to the SQLite database
    :param query: SQL query to execute
    :return: Results of the query
    """
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Close the connection
        conn.close()

        return results
    except sqlite3.Error as e:
        return {"error": f"SQLite error: {str(e)}"}

# Path to the merged GTFS database
db_path = "merged_gtfs.db"

# Example query to fetch all routes from the 'routes' table
query = "SELECT * FROM routes LIMIT 5;"  # You can modify this query as needed

# Run the query
results = test_query(db_path, query)

# Print the results
if isinstance(results, list):
    for row in results:
        print(row)
else:
    print(results)  # In case of an error, print the error message


('5-T01-y08-1', 1, 'T1', None, 0, 'F47922', '000000')
('11-MGr-n-y08-1', 1, 'MGrn', None, 1, '0AAE53', '000000')
('11-MRe-1-y08-1', 1, 'MRed1', None, 1, 'E91D2F', '000000')
('11-MRe-2-y08-1', 1, 'MRed2', None, 1, 'E91D2F', '000000')
('10', 1, '10', 'Gold Souq Bus Stn - Al Quoz Bus Stn', 3, '6F2E90', 'FFFFFF')


In [6]:
import streamlit as st
from tools import GeocodingTool, GTFSCoordinatorTool, CurrentDateTime, RoutesCoordinatorTool
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_openai import ChatOpenAI

gtfs_tool = GTFSCoordinatorTool()
geocode_tool = GeocodingTool()
curr_dt_tool = CurrentDateTime()
route_tool = RoutesCoordinatorTool()

llm = ChatOpenAI(
                base_url="https://api.sambanova.ai/v1/",
                api_key=st.secrets["SAMBANOVA_API_KEY"], 
                streaming=True,
                model="Meta-Llama-3.1-405B-Instruct",
            )

tools = [
    Tool(
        name="GeocodingTool",
        func=geocode_tool.run,
        description="Fetches geocoding information from TomTom's API based on user inputs"
    ),
    Tool(
        name="RouteCoordinatorTool",
        func=route_tool.run,
        description="Fetches route information from Bing Routes API based on user inputs",
    ),
    Tool(
        name="CurrentDateTime",
        func=curr_dt_tool.run,
        description="Returns the current date and time."
    ),
    Tool(
        name="GTFSCoordinatorTool",
        func=gtfs_tool.run,
        description="Handles queries related to public transit data such as schedules, routes, and stop information"
    ),
]

sys_msg = """
        You are an intelligent assistant specializing in public transit for Dubai. Your role is to assist users with queries related to transit schedules, routes, stops, geolocation, and route planning using the following available tools:

        GTFSCoordinatorTool: Accesses non-realtime GTFS data (transit schedules, stops, and routes).
        GeocodingTool: Resolves place names into geographic coordinates.
        CurrentDateTime: Provides current date and time.
        TransitRoutingTool: Generates optimized public transit routes between waypoints.
        Instructions:

        1. Understand the User Query:
        Analyze the user's input to determine the required actions:

        Geolocation data: Resolve place names into coordinates (GeocodingTool).
        Transit data: Retrieve transit schedules, stops, or routes (GTFSCoordinatorTool).
        Time-sensitive data: Determine current date and time (CurrentDateTime).
        Route planning: Generate public transit routes between waypoints (TransitRoutingTool).

        2. Provide Accurate Responses:

        Use GTFSCoordinatorTool to query transit data for routes, stops, or schedules.
        Use GeocodingTool to resolve location names to coordinates.
        Use CurrentDateTime to factor in time-based context (e.g., current date and time).
        Use TransitRoutingTool for optimized route planning between waypoints.

        3. Output Format:
        Provide responses in a structured, clear format:

        Transit details: Schedules, routes, and stop information should be provided clearly.
        Geolocation results: Include coordinates and descriptions for places.
        Time-sensitive data: Include current time context or specified time.
        Route planning: Provide step-by-step directions, estimated travel times, and transit modes.

        4. Stay Contextual:
        Ensure responses are Dubai-specific and reflect local transit services (bus, metro, tram):

        Prioritize public transit in route planning.
        Avoid unnecessary detours and suggest alternatives when possible.
        For routing, always specify both start and end locations as waypoints.
        In time-sensitive queries, factor in the current time for arrivals/departures.
        
        5. Collaborate with Tools:
        Use available tools efficiently. Avoid assumptions about transit data, geolocation, or routing without querying the relevant tool. Ensure that all relevant data is queried from the tools before providing answers.
"""

agent = initialize_agent(
    tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, system_message=sys_msg, handle_parsing_errors=True
)

query = "What's the next bus coming to Dubai Museum?"

response = agent.invoke(query)
print("Response: ", response)




> Entering new AgentExecutor chain...
Thought: To find the next bus coming to Dubai Museum, I need to first get the geocoding information for Dubai Museum, and then use that information to query the public transit data for the next bus arrival.

Action: GeocodingTool
Action Input: {"query": "Dubai Museum"}

Observation: {'summary': {'query': 'query dubai museum', 'queryType': 'NON_NEAR', 'queryTime': 98, 'numResults': 0, 'offset': 0, 'totalResults': 0, 'fuzzyLevel': 2}, 'results': []}
Thought:Thought: It seems like the geocoding information for Dubai Museum is not available. Let me try to rephrase the query to get the geocoding information.

Action: GeocodingTool
Action Input: {"query": "Al Fahidi St, Dubai, UAE"}

Observation: {'summary': {'query': 'query al fahidi st dubai uae', 'queryType': 'NON_NEAR', 'queryTime': 248, 'numResults': 10, 'offset': 0, 'totalResults': 20972, 'fuzzyLevel': 2}, 'results': [{'type': 'Street', 'id': '7cNiobJZzNrkXu9cGsV26w', 'score': 8.6688928604, 'matc

In [91]:
import requests

response = requests.get(f"http://dev.virtualearth.net/REST/V1/Routes/Transit?o=json&wp.0=london&wp.1=leeds&avoid=minimizeTolls&key={st.secrets["BINGMAPS_KEY"]}")
response

<Response [200]>

In [79]:
response.json()['resourceSets'][0]['resources'][0]['travel']

dict_keys(['__type', 'bbox', 'id', 'currencyCode', 'distanceUnit', 'durationUnit', 'routeLegs', 'trafficCongestion', 'trafficDataUsed', 'travelDistance', 'travelDuration', 'travelDurationTraffic', 'travelMode'])

In [81]:
travelDistance = response.json()['resourceSets'][0]['resources'][0]['travelDistance']
travelDuration = response.json()['resourceSets'][0]['resources'][0]['travelDuration']

travelDistance, travelDuration

(276.646, 16188)

In [66]:
start_coords = response.json()['resourceSets'][0]['resources'][0]['routeLegs'][0]['actualStart']
end_coords = response.json()['resourceSets'][0]['resources'][0]['routeLegs'][0]['actualEnd'] 

start_coords, end_coords

({'type': 'Point', 'coordinates': [51.507434, -0.128006]},
 {'type': 'Point', 'coordinates': [53.800686, -1.55029]})

In [68]:
response.json()['resourceSets'][0]

{'estimatedTotal': 1,
 'resources': [{'__type': 'Route:http://schemas.microsoft.com/search/local/ws/rest/v1',
   'bbox': [51.507099, -1.55029, 53.800768, -0.128006],
   'id': 'v70,h-1666434989,i0,a2,cen-US,dAAAAAAAAAAA1,y1,s1,m4,o1,t0,wSN3OvvLASUDqB3WRQlnAvw2~AXf2AverdXoJAADgAQAAgD8A0~TG9uZG9uLCBHcmVhdGVyIExvbmRvbiwgVW5pdGVkIEtpbmdkb201~~~~~~~~v12,w72_QXn3mSkCHqMKf4c34vw2~AXf2AveTsIoHAADgAQAAAAAA0~TGVlZHMsIFdlc3QgWW9ya3NoaXJlLCBVbml0ZWQgS2luZ2RvbQ2~~~~~~~~v12,k1,pZ3NpLXBvaQ2-VHJ1ZQ2',
   'currencyCode': '',
   'distanceUnit': 'Kilometer',
   'durationUnit': 'Second',
   'routeLegs': [{'actualEnd': {'type': 'Point',
      'coordinates': [53.800686, -1.55029]},
     'actualStart': {'type': 'Point', 'coordinates': [51.507434, -0.128006]},
     'alternateVias': [],
     'endLocation': {'bbox': [53.698959, -1.800431, 53.952049, -1.165114],
      'name': 'Leeds, West Yorkshire, United Kingdom',
      'point': {'type': 'Point', 'coordinates': [53.800701, -1.550264]},
      'address': {'adminD

In [69]:
itinerary_items = response.json()['resourceSets'][0]['resources'][0]['routeLegs'][0]['itineraryItems']

instructions = [item['instruction']['text'] for item in itinerary_items] 
instructions

['Walk from London, Greater London, United Kingdom to Trafalgar Square',
 'Bus: 453 - Marylebone',
 'Walk from Baker Street Station to Baker Street Underground Station',
 'Subway: Jubilee - Wembley Park',
 'Walk from Finchley Road Underground Station to Finchley Road Station (Stop CL)',
 'Bus: M20',
 'Walk from Bus Stn (Stand 1) to Leeds, West Yorkshire, United Kingdom']

In [48]:
itinerary_items[0]

{'compassDirection': '',
 'details': [{'endPathIndices': [4],
   'maneuverType': 'Walk',
   'mode': 'Transit',
   'roadType': 'NotApplicable',
   'startPathIndices': [0]}],
 'exit': '',
 'iconType': 'Walk',
 'instruction': {'maneuverType': 'DepartStart',
  'text': 'Walk from London, Greater London, United Kingdom to Trafalgar Square'},
 'isRealTimeTransit': False,
 'maneuverPoint': {'type': 'Point', 'coordinates': [51.507434, -0.128006]},
 'realTimeTransitDelay': 0,
 'sideOfStreet': 'Unknown',
 'tollZone': '',
 'towardsRoadName': '',
 'transitTerminus': '',
 'travelDistance': 0.129,
 'travelDuration': 92,
 'travelMode': 'Transit'}